In [3]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
yolo_model = YOLO("/Users/vamsikasibhotla/Desktop/WOXSEN/Trimester 6/SPORTS ANALYTICS/Project/yolov8n.pt")

class YOLOPlayerDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("YOLOv8 Football Player Detection")

        # Heading (updated)
        tk.Label(root, text="On-Field Football Player Detection", font=("Arial", 24)).pack(pady=10)

        # Buttons (renamed)
        tk.Button(root, text="Upload a Video", command=self.load_video).pack(pady=5)
        tk.Button(root, text="Detect Players", command=self.run_yolo).pack(pady=5)

        # Image display label
        self.image_label = tk.Label(root)
        self.image_label.pack(padx=10, pady=10)

        # Detection count label
        self.count_label = tk.Label(root, text="", font=("Arial", 14))
        self.count_label.pack(pady=5)

        self.video_path = None

    def load_video(self):
        self.video_path = filedialog.askopenfilename(title="Select Football Video", filetypes=[("Video Files", "*.mp4 *.avi *.mov")])
        if self.video_path:
            cap = cv2.VideoCapture(self.video_path)
            success, frame = cap.read()
            cap.release()
            if success:
                self.display_image(frame)
                self.count_label.config(text="")

    def run_yolo(self):
        if not self.video_path:
            return

        cap = cv2.VideoCapture(self.video_path)
        success, frame = cap.read()
        cap.release()

        if success:
            results = yolo_model(frame)

            # Filter only 'person' class (class ID 0)
            person_detections = [d for d in results[0].boxes.data if int(d[5]) == 0]

            player_count = len(person_detections)

            for det in person_detections:
                x1, y1, x2, y2 = map(int, det[:4])
                # Draw slimmer green rectangle without label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

            self.display_image(frame)
            self.count_label.config(text=f"Number of players detected: {player_count}")

    def display_image(self, img):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        img_pil = img_pil.resize((800, 450))
        img_tk = ImageTk.PhotoImage(image=img_pil)

        self.image_label.configure(image=img_tk)
        self.image_label.image = img_tk

if __name__ == "__main__":
    root = tk.Tk()
    app = YOLOPlayerDetectionApp(root)
    root.mainloop()


0: 384x640 18 persons, 63.7ms
Speed: 2.1ms preprocess, 63.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 persons, 65.0ms
Speed: 5.4ms preprocess, 65.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
